In [ ]:
%matplotlib inline
import pandas as pd

In [ ]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [ ]:
import os
tfile = os.path.join("C:\\trader\journal\_08_August\Week_3\_0817_Friday", "Trades.csv")
tfile="TradesWithHolds.csv"
# tfile="TradesExcelEdited.csv"

#The copied path from file explorer fails
t=os.path.join("C:\trader\journal\_08_August\Week_3\_0817_Friday", "Trades.csv")



t

In [ ]:
trades = pd.read_csv(tfile)
trades

In [ ]:
def addRowsOld(dframe, numRow) :
    ll=list()
    r=list()
    for i in range(len(dframe.columns)) :
        r.append('')
        
    for i in range(numRow) :
        ll.append(r)
    newdf= pd.DataFrame(ll, columns=dframe.columns)
    
    try :
        assert(len(ll[0]) == len(dframe.columns))
    except AssertionError as ex :
        print("Coding error. Assertion is untrue:", ex)
        newdf="Data Frame is not initialized"
        return newdf
    
    dframe = dframe.append(newdf, ignore_index=True)
    
    return dframe

    

In [ ]:
def createDf(dframe, numRow) :
    ''' Creates a new DataFrame with  the length numRow. Each cell is filled with empty string '''

    ll=list()
    r=list()
    for i in range(len(dframe.columns)) :
        r.append('')
        
    for i in range(numRow) :
        ll.append(r)
    newdf= pd.DataFrame(ll, columns=dframe.columns)

    return newdf

In [ ]:
def addRows(dframe, numRow) :
    newdf= createDf(dframe, numRow)
        
    
    dframe = dframe.append(newdf, ignore_index=True)
    
    return dframe

    

### Make sure the hour string is 0 padded.  Should probably change these to data types. 

### Todo- This, and the duration, will get more complex with the IB report format. The html report uses a long timedate format

In [ ]:
def zeroPadTimeStr(dframe, timeHeading) :
    '''Guarantee that the time format xx:xx:xx'''
    

    for i, row in dframe.iterrows():
        tm = row[timeHeading]
        tms=tm.split(":")
        if int(len(tms[0]) < 2) :
            if tms[0].startswith("0") == False :
                tm= "0" + tm
                dframe.at[i, timeHeading] = tm
    return dframe
                

### Todo. Doctor an input csv file to include fractional numer of shares for testing. Make it more modular by checking for 'HOLD'. It might be useful in a windowed version with menus to do things seperately.

In [ ]:
def mkShortsNegative(dframe, side, qty) :
    ''' Fix the shares sold to be negative values. Currently assuming DAS values 'B', 'S', 'SS' for 'Side'. 
    '''
    for i, row in dframe.iterrows():
        if row[side] != 'B' and row[qty] > 0:
            dframe.at[i, qty] = ((dframe.at[i, qty]) * -1)
    return dframe

### def getListTickerDF(df) will take a dataframe that includes tickers in column 'Symb' and returns a python list of DataFrames , 1 for each ticker.

In [ ]:
def getListTickerDF(dframe, tickCol = 'Symb') :
    ldf_tick = list()
    for ticker in dframe[tickCol].unique() :
        ldf = dframe[dframe[tickCol] == ticker]
        ldf_tick.append(ldf)
    return ldf_tick

In [ ]:

def getOvernightTrades(dframe, tickCol='Symb', qtyCol='Qty') :
    ''' getOvernightTrades(dframe) takes the DataFrame and returns a list of lists 
        (symb, qty) of trades that have overnight shares. '''
    
    ldf_tick = getListTickerDF(dframe, tickCol)
    overnightTrade = list()
    for ticker in ldf_tick :
        if ticker[qtyCol].sum() != 0 :
            overnightTrade.append([ticker[tickCol].unique()[0], ticker[qtyCol].sum(), 0, 0])
    return overnightTrade

### Note that this does not yet include those shares that are held before the days trading began. Redo this to remake the list of data frames from the Symbols of Swing List then make a list of data frams that  excludes those then merge them together

In [ ]:
def askUser(st, question, ix, default) :
    while True :
        try :
            response = input(question)
            if len(response) < 1 :
                response = default
            else :
                response = int(response)
        except Exception as ex:
            print(ex)
            print("please enter a number")
            continue
       
        st[ix] = response
        return st


In [ ]:
# for i in range(len(swingTrade)) :
def figureOvernightTransactions() :
    swingTrade = getOvernightTrades(trades)
    for i in range(len(swingTrade)) :
        tryAgain =  True
        while tryAgain == True :

            xticker = 0      #Candy coaters
            xbalance = 1
            xbefore = 2
            xafter = 3
            print(swingTrade[i])
            print ("There is an unbalanced amount of shares of {0} in the amount of {1}".format(swingTrade[i][xticker], swingTrade[i][xbalance]))

            question = "How many shares of {0} are you holding now? (Enter for {1})".format(swingTrade[i][xticker], swingTrade[i][xbalance])
            swingTrade[i] = askUser(swingTrade[i], question, xafter, swingTrade[i][xbalance])

            if swingTrade[i][xafter] != swingTrade[i][xbalance]:

                difference = swingTrade[i][xbalance] - swingTrade[i][xafter]
                statement = "There is now a prior unbalanced amount of shares of {0} in the amount of {1}"
                print(statement.format(swingTrade[i][xticker], difference))
                question = "How many shares of {0} were you holding before?".format(swingTrade[i][xticker])
                swingTrade[i] = askUser(swingTrade[i], question, xbefore, difference)

            unaccounted = swingTrade[i][xbefore] + swingTrade[i][xafter] - swingTrade[i][xbalance]
            if unaccounted == 0 :
                print("That works.")
                tryAgain = False
            else :
                print()
                print("There are {1} unaccounted for shares in {0}".format(swingTrade[i][xticker], unaccounted))
                print()
                print("That does not add up. Starting over ...")
                print()
                print ("Prior to reset version ", i, swingTrade)
                swingTrade[i] = getOvernightTrades(trades)[i]
                print ("reset version ", i, swingTrade)
    return swingTrade

In [ ]:
def insertOvernightRowold(dframe, swingTrade, time='Time', symbol='Symb', side='Side', price='Price', qty='Qty', acct="Account", PL='P / L') :
    newdf = createDf(trades, 0)
    for ldf in getListTickerDF(dframe) :
        found=False
        for ticker, balance, before, after in swingTrade:
            if ticker == ldf.Symb.unique()[0] :
#                 print ("Got {0} with the balance {1}". format (ticker, balance))
                found = True
                ldf = addRows(ldf, 1)
                for i, row in ldf.iterrows():

                    if i == len(ldf) -1 :
                        ldf.at[i, time] = '23:59:59'
                        ldf.at[i, symbol] = ticker
                        if balance > 0 :
                            ldf.at[i, side] = "HOLD+"
                        else :
                            ldf.at[i, side] = "HOLD-"
                        ldf.at[i, price] = 0
                        ldf.at[i, qty] = after
                        ldf.at[i, acct] = 'ZeroSubstance'
                        ldf.at[i, PL] = 0
        newdf = newdf.append(ldf, ignore_index = True)
    return newdf

In [ ]:
def insertOvernightRow(dframe, st, time='Time', symbol='Symb', side='Side', price='Price', qty='Qty', acct="Account", PL='P / L') :
    newdf = createDf(trades, 0)
    
    for ldf in getListTickerDF(dframe) :
        found=False
        for ticker, balance, before, after in swingTrade:
            if ticker == ldf.Symb.unique()[0] :
                print ("Got {0} with the balance {1}, before {2} and after {3}". format (ticker, balance, before, after))
                if before != 0 :
                    newldf = createDf(trades, 1)
                    print("length:   ", len(newldf))
                    for i, row in newldf.iterrows():

                        if i == len(newldf) -1 :
                            print("Though this seems unnecessary it will make it more uniform ")
                            newldf.at[i, time] = '00:00:01'
                            newldf.at[i, symbol] = ticker
                            if before > 0 :
                                newldf.at[i, side] = "HOLD+"
                            else :
                                newldf.at[i, side] = "HOLD-"
                            newldf.at[i, price] = 0
                            newldf.at[i, qty] = before
                            newldf.at[i, acct] = 'ZeroSubstance'
                            newldf.at[i, PL] = 0
                            
                            ldf = newldf.append(ldf, ignore_index = True)
                        break #This should be unnecessary as newldf should always be the length of 1 here
                if after != 0 :
                    print("Are we good?")
                    ldf = addRows(ldf, 1)
        
                    for i, row in ldf.iterrows():

                        if i == len(ldf) -1 :
                            ldf.at[i, time] = '23:59:59'
                            ldf.at[i, symbol] = ticker
                            if balance > 0 :
                                ldf.at[i, side] = "HOLD+"
                            else :
                                ldf.at[i, side] = "HOLD-"
                            ldf.at[i, price] = 0
                            ldf.at[i, qty] = after
                            ldf.at[i, acct] = 'ZeroSubstance'
                            ldf.at[i, PL] = 0
        
        newdf = newdf.append(ldf, ignore_index = True)
    return newdf

# Notebook visual tester code

In [ ]:
# ddf = createDf(trades, 0)
# ddf

In [ ]:
# x = zeroPadTimeStr(trades, 'Time')
# x.head()

In [ ]:
# trades

In [ ]:
# x = mkShortsNegative(trades, 'Side', 'Qty')
# x

In [ ]:
# x = getListTickerDF(trades)
# x[2]

In [ ]:
# st = getOvernightTrades(trades)

In [ ]:
# st

In [ ]:
# st= figureOvernightTransactions()

In [ ]:
# st

In [ ]:
# x = insertOvernightRow(trades, st)
# x
                    
                    

# End visual tester code

In [ ]:
trades = zeroPadTimeStr(trades, 'Time')

In [ ]:
trades.head()       
   

In [ ]:
trades = trades.sort_values(['Symb', 'Time'])

In [ ]:
# trades


In [ ]:
trades = mkShortsNegative(trades, 'Side', 'Qty')

In [ ]:
trades.head()

### Check to see if any trades are either open now or missing the opening transaction from a preveious time. If there are, Place (ticker, numshares) tuple in a list swingTrades.

In [ ]:
swingTrade = getOvernightTrades(trades)

In [ ]:
swingTrade

In [ ]:
len(swingTrade)

In [ ]:
swingTrade

### For each ticker that has a balance o f shares, ask the user for informatiion. If the reslting numbers of shares is accounted for great if not, add a row in the ticker df for prior transactions  and one for shares still holding.
### This will be a dialogue where the user adjusts share numbers until we geto to the correct balance.

###### Start here. This code is all messed up

In [ ]:
swingTrade = figureOvernightTransactions()

In [ ]:
print(swingTrade)

In [ ]:
swingTrade

In [ ]:
listdf = insertOvernightRow(trades, swingTrade)
listdf

In [ ]:
lbls = ['Tindex', 'Start', 'Time', 'Symb', 'Side', 'Price', 'Qty','Balance', 'Account', "P / L", 'Sum', 'Duration', 'Name']

In [ ]:
for l in lbls :
    if l not in listdf.columns :
        listdf[l] = ''
# trades.columns
newTrades = listdf[lbls]
newTrades.copy()
newTrades.columns

In [ ]:
newTrades

In [ ]:
nt = newTrades.sort_values(['Symb', 'Time'])
nt.head()

In [ ]:
def writeShareBalance(dframe) :
    prevBal = 0
    for i, row in dframe.iterrows():
        qty = (dframe.at[i,'Qty'])
        if row['Side'].startswith("HOLD") :
            print("got it at ", qty)
            qty = qty * -1
        newBalance = qty + prevBal

        dframe.at[i,'Balance'] = newBalance 
        prevBal = newBalance    
    return nt

In [ ]:
nt = writeShareBalance(nt)
nt

In [ ]:
# prevBal = 0
# for i, row in nt.iterrows():
#     qty = (nt.at[i,'Qty'])
#     if row['Side'].startswith("HOLD") :
#         print("got it at ", qty)
#         qty = qty * -1
#     newBalance = qty + prevBal
    
#     nt.at[i,'Balance'] = newBalance 
#     prevBal = newBalance                
    
# # nt

In [ ]:
nt

In [ ]:
def addStartTime(dframe) :
    newTrade = True
    for i, row in dframe.iterrows():
        if newTrade :
            if row['Side'].startswith('HOLD') and i < len(dframe):
                print("got Hold and the next time is {0}, {1}, {2}".format(dframe.at[i+1, 'Time'], i, len(dframe)))
                dframe.at[i, 'Start'] = dframe.at[i+1, 'Time']
            else :
                dframe.at[i,'Start'] = row['Time']
            newTrade = False
            oldTime = row['Start']
        else :
            dframe.at[i, 'Start'] = oldTime
        if row['Balance'] == 0 :
            newTrade = True
    return dframe
        

In [ ]:
nt = addStartTime(nt)
nt

In [ ]:
nt = nt.sort_values(['Start', 'Time'])
nt

## Now that we have Trade index and start times everything else in this file should use filters nt[nt.Tindex == "Trade 1"] . This way each trade can be treated as an object that includes a variable number of transactions.

In [ ]:
def addTradeIndex(dframe) :

    TCount = 1
    prevEndTrade = -1

    for i, row in dframe.iterrows():
        if len(row['Symb']) < 1 :
            break
        tradeIndex = "Trade " + str(TCount)
        if prevEndTrade == 0:
            TCount = TCount + 1
            prevEndTrade = -1
        tradeIndex = "Trade " + str(TCount)
        dframe.at[i,'Tindex'] = tradeIndex 
        if row['Balance'] == 0 :
            prevEndTrade = 0
    numTrades = TCount
    print(numTrades)        
    return dframe

In [ ]:
# #def addTradeindex
# TCount = 1
# prevEndTrade = -1

# for i, row in nt.iterrows():
#     if len(row['Symb']) < 1 :
#         break
#     tradeIndex = "Trade " + str(TCount)
#     if prevEndTrade == 0:
#         TCount = TCount + 1
#         prevEndTrade = -1
#     tradeIndex = "Trade " + str(TCount)
#     nt.at[i,'Tindex'] = tradeIndex 
#     if row['Balance'] == 0 :
#         prevEndTrade = 0
# numTrades = TCount
# print(numTrades)        
# nt

In [ ]:
nt = addTradeIndex(nt)
nt.head()

In [ ]:
def addTradePL (dframe) :
    tradeTotal = 0.0
    for i, row in dframe.iterrows():
        if row['Balance'] != 0 :
            tradeTotal = tradeTotal + row['P / L']
        else :
            sumtotal = tradeTotal + row['P / L']
            dframe.at[i, 'Sum'] = sumtotal
            tradeTotal = 0
    return dframe
nt = addTradePL(nt)
nt.head()

In [ ]:
# #Use filter instead of iteration
# # def addTradePL
# tradeTotal = 0.0
# for i, row in nt.iterrows():
#     if row['Balance'] != 0 :
#         tradeTotal = tradeTotal + row['P / L']
#     else :
#         sumtotal = tradeTotal + row['P / L']
#         nt.at[i, 'Sum'] = sumtotal
#         tradeTotal = 0
# nt.head()

In [ ]:
import datetime

In [ ]:
def addTradeDuration(dframe) :
    
    tradeTotal = 0.0
    for i, row in dframe.iterrows():
        if row['Balance'] == 0 :
            timeEnd = row['Time']
            timeStart = row['Start']
            end=timeEnd.split(":")
            start=timeStart.split(":")
            diff = datetime.datetime(1,1,1,int(end[0]), int(end[1]), int(end[2])) - datetime.datetime(1,1,1,int(start[0]), int(start[1]), int(start[2]))
            dframe.at[i,'Duration'] = diff
    return dframe
nt = addTradeDuration(nt)
nt.head()

In [ ]:
# #Use filter instead of iteration
# # def addTradeDuration 
# tradeTotal = 0.0
# for i, row in nt.iterrows():
#     if row['Balance'] == 0 :
#         timeEnd = row['Time']
#         timeStart = row['Start']
#         end=timeEnd.split(":")
#         start=timeStart.split(":")
#         diff = datetime.datetime(1,1,1,int(end[0]), int(end[1]), int(end[2])) - datetime.datetime(1,1,1,int(start[0]), int(start[1]), int(start[2]))
#         nt.at[i,'Duration'] = diff
# nt.head()

In [ ]:
def addTradeName(dframe) :
    for i, row in dframe.iterrows():
        longShort = " Long"
        if row['Balance'] == 0 :
            if row['Side'] == 'B' :
                longShort = " Short"
            dframe.at[i, 'Name'] = row['Symb'] + longShort
    return dframe

nt = addTradeName(nt)
nt.head()

In [ ]:
# # def addTradeName
# for i, row in nt.iterrows():
#     longShort = " Long"
#     if row['Balance'] == 0 :
#         if row['Side'] == 'B' :
#             longShort = " Short"
#         nt.at[i, 'Name'] = row['Symb'] + longShort
# nt.head()

In [ ]:
nt=addRows(nt,1)
nt.tail(10)

### This will blow up from wrong types if there is some kind of anomaly. The blank rows in Sum col are str and the filled rows are float. Don't fix it until it blows up. (or you need change this 'now' to production code for some reason) That way we have some case to fix.
### Also need to fix below if the lastCol for Balance > 0 instead of what's there

In [ ]:
# nt['P / L'].sum()
nt['P / L'].dtype

In [ ]:
nt

In [ ]:
# Note that .sum() should work on this but it failed when I tried it.
def addSummaryPL(dframe) :
    
    count=0
    tot=0.0
    tot2 = 0.0
    for i, row in dframe.iterrows():
        count=count+1
        if count < len(dframe) :
            tot=tot+row['P / L']
            if row['Balance'] == 0 :
 
                tot2 = tot2 + row['Sum']
            if count == len(dframe) -1 :
                lastCol = row['P / L']

                print("Last col?", row['P / L'])
   
        else :
            dframe.at[i, 'P / L'] = tot
            dframe.at[i, 'Sum'] = tot2

    if lastCol > 0:

        print('''
        Some shares are unaccounted for. Please send the original csv file to the developer in 
        order to fix ths issue in the software. Please remove the account number  or change its value
        to anything else.
        ''')
    return dframe
nt = addSummaryPL(nt)
# nt = addSummaryPL(nt)
nt



In [ ]:
# # def addSummaryPL
# count=0
# tot=0.0
# tot2 = 0.0
# for i, row in nt.iterrows():
#     count=count+1
#     if count < len(nt) :
#         tot=tot+row['P / L']
#         if row['Balance'] == 0 :
# #             assert(type(row['Sum']) == type(1.1))
#             tot2 = tot2 + row['Sum']
#         if count == len(nt) -1 :
#             lastCol = row['P / L']
# #             assert(type(lastCol) == type(1.1))
#             print("Last col?", row['P / L'])
# #         print(count, ": ", tot, tot2)
#     else :
#         nt.at[i, 'P / L'] = tot
#         nt.at[i, 'Sum'] = tot2
# #         print("Grand total: ", tot, tot2)

# # if(tot != tot2) or lastCol > 0:

# if lastCol > 0:

#     print('''
#     Some shares are unaccounted for. Please send the original csv file to the developer in 
#     order to fix ths issue in the software. Please remove the account number  or change its value
#     to anything else.
#     ''')
# nt



In [ ]:
nt

In [ ]:

# Now  we are going to add each trade and insert space to put in pictures with circles and arrows and 
# paragraph on the back of each one to be used as evidence against you in a court of law (or court of bb opionion)
dframe = nt
ldf = list()
count = 1
while True :
    tradeStr = "Trade " + str(count)
    count = count + 1
    tdf = dframe[dframe.Tindex == tradeStr]
    if len(tdf) > 0 :
        ldf.append(tdf)
    else :
        break
len(ldf)

dframe = addRows(dframe, 2)

for tdf in ldf :
    dframe = dframe.append(tdf, ignore_index = True)
    dframe = addRows(dframe, 20)
    print(count, ": ", len(dframe))
    
nt = dframe
nt

In [ ]:
nt

In [ ]:

# # Now  we are going to add each trade and insert space to put in pictures with circles and arrows and 
# # paragraph on the back of each one to be used as evidence against you in a court of law (or court of bb opionion)

# ldf = list()
# count = 1
# while True :
#     tradeStr = "Trade " + str(count)
#     count = count + 1
#     tdf = nt[nt.Tindex == tradeStr]
#     if len(tdf) > 0 :
#         ldf.append(tdf)
#     else :
#         break
# len(ldf)

# nt = addRows(nt, 2)

# for tdf in ldf :
#     nt = nt.append(tdf, ignore_index = True)
#     nt = addRows(nt, 20)
#     print(count, ": ", len(nt))
    
# nt

In [ ]:
nt

In [ ]:
name=datetime.date.today().strftime("Trades_%A_%m%d.csv")
nt.to_csv(name)